# Geospatial Visualization - District

## Geojson Data
  
Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import folium as flm
import calendar
#  For showing all columns in Pandas
pd.set_option('display.max_columns', None)

# this ignores the depreciation warnings etc
import warnings
warnings.filterwarnings("ignore")

### Create a Dataframe containing geometry of the Police Station Areas
  
Read in the data and create a DataFrame.

In [ ]:
# Read the geoJSON file using geopandas
geo_stat = gpd.read_file(r'../../../data/geodata/South_African_police_boundaries.geojson')
geo_stat = geo_stat[["FID", "COMPNT_NM", "geometry"]] # only select 'COMPNT_NM' (Police Stations) and 'geometry' columns

In [ ]:
geo_stat

In [ ]:
geo_stat.info()

Lets check the column names.

In [ ]:
geo_stat.columns

We will renmane the column 'ADM2_EN' to 'district' and 'geometry' to 'geometry_dist'  'COMPNT_NM': 'station', 

In [ ]:
geo_stat.rename(columns = {'geometry': 'geometry_stat'}, inplace = True)
geo_stat

### Create a Dataframe containing the Police Station Crime and Weather Data
  
Read in the data and create a DataFrame.

In [ ]:
df = pd.read_parquet('../../../data/weather/weather_location_crime.parquet')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df = df[
    ['station', 'municipality', 'district', 'province',
     'crime_category', 'date', 'month', 'year',
     'number_of_crimes', 'average_max_temp',
     'average_min_temp', 'average_rain_mm',
     'average_windspeed', 'latitude', 'longitude']]
df.head(1)

Lets check the shape of the dataframes and the length of the 'stations'.  
  
Quick check of the shape reveals there may be some missing data points.

In [ ]:
df.shape, len(df['station'].unique()), df.shape, len(geo_stat['COMPNT_NM'])

We need to match the length of the 'station' column to the 'geo_stat' 'COMP_NM' column.

In [ ]:
df['station'] = df['station'].replace('Bohlokong','Bethlehem')
df['station'] = df['station'].replace('Int Airport C Town', 'Cape Town Central')
df['station'] = df['station'].replace('Pholile', 'Matatiele')
df['station'] = df['station'].replace('Qhasa', 'Flagstaff')
df['station'] = df['station'].replace('Protea', 'Protea Glen')
df['station'] = df['station'].replace('Samora Machel', 'Philippi')

In [ ]:
df.shape, len(df['station'].unique()), df.shape, len(geo_stat['COMPNT_NM'])

Lets create a dictionary to map the stations.

In [ ]:
dict_dist = dict(zip(geo_stat.COMPNT_NM.str.title(), geo_stat.FID))

Create a dictionary of the missing values to map to existing stations.  
  
And update dictionary.

In [ ]:
# dict_new = {
#     'Bohlokong': 275, 'Int Airport C Town': 510,
#     'Pholile': 885, 'Protea': 429, 'Qhasa': 176,
#     'Samora Machel': 117}

In [ ]:
# dict_dist.update(dict_new)

In [ ]:
dict_dist

Then we will map the values and create a new column 'stat_id'. We will use this later to map the geometry data.

In [ ]:
df['stat_id'] = df['station'].map(dict_dist)

Quick check.

In [ ]:
df.sample(n=10)

Now we can check the data types.

In [ ]:
geo_stat.info(show_counts=True)

In [ ]:
df.info(show_counts=True)

In [ ]:
geo_stat['stat_id'] = geo_stat['FID']
geo_stat

## Merge the DataFrames

Lets create a new DataFrame of the merged DataFrames.

In [ ]:
geospatial_stat = geo_stat.merge(df, on=['stat_id'], how='left').fillna('')

In [ ]:
geospatial_stat

Convert to a GeoPandas DataFrame.

In [ ]:
geospatial_stat = gpd.GeoDataFrame(geospatial_stat, geometry='geometry_stat')

In [ ]:
type(geospatial_stat)

Reorder columns

In [ ]:
geospatial_stat.columns

In [ ]:

geospatial_stat = geospatial_stat[
    ['station', 'stat_id', 'district', 'municipality',
     'province', 'crime_category', 'date', 'month',
     'year', 'number_of_crimes', 'average_max_temp',
     'average_min_temp', 'average_rain_mm',
     'average_windspeed', 'latitude', 'longitude',
     'FID', 'COMPNT_NM', 'geometry_stat']]
geospatial_stat.head(1)

In [ ]:
geospatial_stat['year'].info()

Lets change the year to a string.

In [ ]:
geospatial_stat['year'] = geospatial_stat['year'].map(str)

In [ ]:
geospatial_stat['year'].info()

## Mapping
Lets create a dataframe for the Folium map.

### Yearly Data

In [ ]:
map_stat = geospatial_stat.groupby(['stat_id', 'station', 'province', 'district', 'year', 'FID', 'COMPNT_NM'], as_index=False).agg({'number_of_crimes': 'sum'})
map_stat.head()

In [ ]:
map_stat.info()

In [ ]:
geo_stat

Merge Geometry into DataFrame.

In [ ]:
mapped_stat = pd.merge(map_stat, geo_stat, on=['FID', 'stat_id'], how='left')
mapped_stat = mapped_stat[['station', 'stat_id', 'province', 'district', 'year', 'FID', 'number_of_crimes', 'geometry_stat']]
mapped_stat

In [ ]:
mapped_stat.info(show_counts=True)

Convert to a GeoPandas DatFrame.

In [ ]:
mapped_stat = gpd.GeoDataFrame(mapped_stat, geometry='geometry_stat')
type(mapped_stat)

In [ ]:
sa_stat_map = flm.Map(location=[-28.343, 25.862], zoom_start=6, scrollWheelZoom=False, overlay=False, tiles=None)

flm.TileLayer('openstreetmap',name="Light Map",control=False).add_to(sa_stat_map)

ft_2016 = mapped_stat[mapped_stat['year'] == '2016']
ft_2017 = mapped_stat[mapped_stat['year'] == '2017']
ft_2018 = mapped_stat[mapped_stat['year'] == '2018']
ft_2019 = mapped_stat[mapped_stat['year'] == '2019']
ft_2020 = mapped_stat[mapped_stat['year'] == '2020']
ft_2021 = mapped_stat[mapped_stat['year'] == '2021']

fg0 = flm.FeatureGroup(name='ft_2016',overlay=False).add_to(sa_stat_map)
fg1 = flm.FeatureGroup(name='ft_2017',overlay=False).add_to(sa_stat_map)
fg2 = flm.FeatureGroup(name='ft_2018',overlay=False).add_to(sa_stat_map)
fg3 = flm.FeatureGroup(name='ft_2019',overlay=False).add_to(sa_stat_map)
fg4 = flm.FeatureGroup(name='ft_2020',overlay=False).add_to(sa_stat_map)
fg5 = flm.FeatureGroup(name='ft_2021',overlay=False).add_to(sa_stat_map)

# fg1 = flm.FeatureGroup(name='Crimes Per Year', overlay=False).add_to(sa_stat_map)

fs = [fg0, fg1, fg2, fg3, fg4, fg5]
year_data = [ft_2016, ft_2017, ft_2018, ft_2019, ft_2020, ft_2021]


custom_scale = (mapped_stat['number_of_crimes'].quantile((0,0.2,0.4,0.6,0.7,0.8,0.9,1))).tolist()

for i in range(len(year_data)):
    crimes_per_year = flm.Choropleth(
                geo_data=r'../../../data/geodata/South_African_police_boundaries.geojson',
                data=year_data[i],
                columns=['stat_id', 'number_of_crimes'],
                key_on='feature.properties.FID',
                threshold_scale=custom_scale,
                fill_color='YlGnBu',
                nan_fill_color="blue",
                fill_opacity=0.5,
                line_opacity=0.2,
                legend_name='Number of Crimes ',
                highlight=True,
                line_color='black').geojson.add_to(fs[i])

    # Add customized tooltips to the map
    flm.features.GeoJson(
                        data = year_data[i],
                        name='Crimes Per Year',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=flm.features.GeoJsonTooltip(
                            fields=[
                                'province',
                                'district',
                                'station',
                                'year',
                                'number_of_crimes'],
                            aliases=[
                                "Province:",
                                "District:",
                                "Station:",
                                "Year:",
                                "Number of Crimes:"],
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(crimes_per_year)

flm.TileLayer('openstreetmap', overlay=True, name="light mode").add_to(sa_stat_map)
flm.LayerControl(collapsed=False).add_to(sa_stat_map)
sa_stat_map.save('SA_Yearly_Crime_by_District.html')
sa_stat_map